In [1]:
pip install opencv-contrib-python --user


  Obtaining dependency information for opencv-contrib-python from https://files.pythonhosted.org/packages/a7/9e/7110d2c5d543ab03b9581dbb1f8e2429863e44e0c9b4960b766f230c1279/opencv_contrib_python-4.10.0.84-cp37-abi3-win_amd64.whl.metadata
  Using cached opencv_contrib_python-4.10.0.84-cp37-abi3-win_amd64.whl.metadata (20 kB)
Using cached opencv_contrib_python-4.10.0.84-cp37-abi3-win_amd64.whl (45.5 MB)
Note: you may need to restart the kernel to use updated packages.


In [12]:
import os
import cv2
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split


IMG_WIDTH, IMG_HEIGHT = 128, 128
BATCH_SIZE = 32
EPOCHS = 20


def load_dataset(directory):
    images = []
    labels = []
    for label in os.listdir(directory):
        label_dir = os.path.join(directory, label)
        if os.path.isdir(label_dir):
            for image_name in os.listdir(label_dir):
                image_path = os.path.join(label_dir, image_name)
                image = cv2.imread(image_path)
                image = cv2.resize(image, (IMG_WIDTH, IMG_HEIGHT))
                images.append(image)
                labels.append(label)
    return np.array(images), np.array(labels)

def preprocess_dataset(images, labels):
    images = images / 255.0  
    label_encoder = LabelEncoder()
    labels = label_encoder.fit_transform(labels)
    labels = tf.keras.utils.to_categorical(labels)
    return images, labels, label_encoder


def build_model():
    model = Sequential([
        Conv2D(32, (3, 3), activation='relu', input_shape=(IMG_WIDTH, IMG_HEIGHT, 3)),
        MaxPooling2D((2, 2)),
        Conv2D(64, (3, 3), activation='relu'),
        MaxPooling2D((2, 2)),
        Flatten(),
        Dense(128, activation='relu'),
        Dense(len(label_encoder.classes_), activation='softmax')
    ])
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model


dataset_dir = 'C:/Users/jasha/OneDrive/Desktop/New folder (2)'
images, labels = load_dataset(dataset_dir)
images, labels, label_encoder = preprocess_dataset(images, labels)


X_train, X_val, y_train, y_val = train_test_split(images, labels, test_size=0.2, random_state=42)


model = build_model()
history = model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=EPOCHS, batch_size=BATCH_SIZE)


model.save('face_recognition_model.h5')
np.save('label_encoder_classes.npy', label_encoder.classes_)


J:\anaconda\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/20
6/6 ━━━━━━━━━━━━━━━━━━━━ 7s 704ms/step - accuracy: 0.8237 - loss: 0.3157 - val_accuracy: 0.9792 - val_loss: 0.1593
Epoch 2/20
6/6 ━━━━━━━━━━━━━━━━━━━━ 4s 607ms/step - accuracy: 0.9691 - loss: 0.1648 - val_accuracy: 0.5000 - val_loss: 1.8585
Epoch 3/20
6/6 ━━━━━━━━━━━━━━━━━━━━ 4s 618ms/step - accuracy: 0.8281 - loss: 0.6577 - val_accuracy: 0.9792 - val_loss: 0.1196
Epoch 4/20
6/6 ━━━━━━━━━━━━━━━━━━━━ 6s 608ms/step - accuracy: 1.0000 - loss: 0.0100 - val_accuracy: 1.0000 - val_loss: 0.0472
Epoch 5/20
6/6 ━━━━━━━━━━━━━━━━━━━━ 4s 581ms/step - accuracy: 1.0000 - loss: 0.0166 - val_accuracy: 1.0000 - val_loss: 0.0000e+00
Epoch 6/20
6/6 ━━━━━━━━━━━━━━━━━━━━ 4s 617ms/step - accuracy: 1.0000 - loss: 0.0000e+00 - val_accuracy: 1.0000 - val_loss: 0.0000e+00
Epoch 7/20
6/6 ━━━━━━━━━━━━━━━━━━━━ 4s 595ms/step - accuracy: 1.0000 - loss: 0.0000e+00 - val_accuracy: 1.0000 - val_loss: 0.0000e+00
Epoch 8/20
6/6 ━━━━━━━━━━━━━━━━━━━━ 4s 653ms/step - accuracy: 1.0000 - loss: 2.4924e-09 - val_accu

In [16]:
import cv2
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import load_model


IMG_WIDTH, IMG_HEIGHT = 128, 128


model = load_model('face_recognition_model.h5')
label_encoder_classes = np.load('label_encoder_classes.npy')

def preprocess_image(image_path):
    image = cv2.imread(image_path)
    image = cv2.resize(image, (IMG_WIDTH, IMG_HEIGHT))
    image = image / 255.0
    image = np.expand_dims(image, axis=0)
    return image


def recognize_face(image_path):
    image = preprocess_image(image_path)
    predictions = model.predict(image)
    predicted_label = np.argmax(predictions)
    predicted_class = label_encoder_classes[predicted_label]
    return predicted_class


image_path = 'C:/Users/jasha/OneDrive/Desktop/New folder (2)/person2/rdj-one_one - Copy (11) - Copy.webp'
recognized_person = recognize_face(image_path)
print(f'Recognized Person: {recognized_person}')


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 143ms/step
Recognized Person: person2
